### This notebook is Final submission for the hackathon 

https://www.machinehack.com/hackathons/predict_the_flight_ticket_price_hackathon/overview

Alert: This notebook is an old notebook so I couldn't structure the preprocessing and model code.

In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingRegressor,RandomForestRegressor,ExtraTreesRegressor
import xgboost as xgb
import lightgbm as lgbm
import seaborn as sns

/anaconda3/lib/python2.7/site-packages/lightgbm/__init__.py:46: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [424]:
data = pd.read_excel('Data_Train.xlsx')

In [425]:
data

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302
5,SpiceJet,24/06/2019,Kolkata,Banglore,CCU → BLR,09:00,11:25,2h 25m,non-stop,No info,3873
6,Jet Airways,12/03/2019,Banglore,New Delhi,BLR → BOM → DEL,18:55,10:25 13 Mar,15h 30m,1 stop,In-flight meal not included,11087
7,Jet Airways,01/03/2019,Banglore,New Delhi,BLR → BOM → DEL,08:00,05:05 02 Mar,21h 5m,1 stop,No info,22270
8,Jet Airways,12/03/2019,Banglore,New Delhi,BLR → BOM → DEL,08:55,10:25 13 Mar,25h 30m,1 stop,In-flight meal not included,11087
9,Multiple carriers,27/05/2019,Delhi,Cochin,DEL → BOM → COK,11:25,19:15,7h 50m,1 stop,No info,8625


In [426]:
data.Additional_Info.unique()

array([u'No info', u'In-flight meal not included',
       u'No check-in baggage included', u'1 Short layover', u'No Info',
       u'1 Long layover', u'Change airports', u'Business class',
       u'Red-eye flight', u'2 Long layover'], dtype=object)

In [427]:
data.Route.nunique()

128

In [428]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10683 entries, 0 to 10682
Data columns (total 11 columns):
Airline            10683 non-null object
Date_of_Journey    10683 non-null object
Source             10683 non-null object
Destination        10683 non-null object
Route              10682 non-null object
Dep_Time           10683 non-null object
Arrival_Time       10683 non-null object
Duration           10683 non-null object
Total_Stops        10682 non-null object
Additional_Info    10683 non-null object
Price              10683 non-null int64
dtypes: int64(1), object(10)
memory usage: 918.1+ KB


In [429]:
data.columns

Index([u'Airline', u'Date_of_Journey', u'Source', u'Destination', u'Route',
       u'Dep_Time', u'Arrival_Time', u'Duration', u'Total_Stops',
       u'Additional_Info', u'Price'],
      dtype='object')

In [430]:
data.Total_Stops.unique()

array([u'non-stop', u'2 stops', u'1 stop', u'3 stops', nan, u'4 stops'],
      dtype=object)

In [431]:
test_data = pd.read_excel('Test_set.xlsx')

In [432]:
test_data.Route.nunique()

100

In [433]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2671 entries, 0 to 2670
Data columns (total 10 columns):
Airline            2671 non-null object
Date_of_Journey    2671 non-null object
Source             2671 non-null object
Destination        2671 non-null object
Route              2671 non-null object
Dep_Time           2671 non-null object
Arrival_Time       2671 non-null object
Duration           2671 non-null object
Total_Stops        2671 non-null object
Additional_Info    2671 non-null object
dtypes: object(10)
memory usage: 208.7+ KB


In [434]:
data.Airline.unique()

array([u'IndiGo', u'Air India', u'Jet Airways', u'SpiceJet',
       u'Multiple carriers', u'GoAir', u'Vistara', u'Air Asia',
       u'Vistara Premium economy', u'Jet Airways Business',
       u'Multiple carriers Premium economy', u'Trujet'], dtype=object)

In [435]:
data.Arrival_Time.nunique()

1343

In [436]:
target = data.Price

In [437]:
train_data = data[['Airline', 'Date_of_Journey', 'Source', 'Destination', 'Route',
             'Dep_Time', 'Arrival_Time', 'Duration', 'Total_Stops', 'Additional_Info']]

In [438]:
all_data = train_data.append(test_data, sort=False, ignore_index=True)

In [439]:
all_data.loc[10683]

Airline                Jet Airways
Date_of_Journey          6/06/2019
Source                       Delhi
Destination                 Cochin
Route              DEL → BOM → COK
Dep_Time                     17:30
Arrival_Time          04:25 07 Jun
Duration                   10h 55m
Total_Stops                 1 stop
Additional_Info            No info
Name: 10683, dtype: object

In [440]:
test_data.loc[0]

Airline                Jet Airways
Date_of_Journey          6/06/2019
Source                       Delhi
Destination                 Cochin
Route              DEL → BOM → COK
Dep_Time                     17:30
Arrival_Time          04:25 07 Jun
Duration                   10h 55m
Total_Stops                 1 stop
Additional_Info            No info
Name: 0, dtype: object

In [443]:
all_data.loc[df2=='nan','Total_Stops'] = '1 stop'
all_data.loc[df2=='nan','Route'] = 'DEL → MAA → COK' 

In [444]:
"""
df2 = df["Address"].apply(lambda x: x.split(","))
city = df2.apply(lambda x: " ".join(x[0].split()[1:]))
state = df2.apply(lambda x: x[1].split()[0])
result = pd.DataFrame(zip(city, state), columns=["city", "state"])
"""
df2 = all_data['Total_Stops'].apply(lambda x: str(x).split(" "))
df2 = df2.apply(lambda x: "".join(x[0].split()))
df2 = df2.replace('non-stop','0')

df2.value_counts()

#all_data['Total_Stops'] = df2.astype(int) 

In [445]:
all_data['Total_Stops'].value_counts()


1    7057
0    4340
2    1899
3      56
4       2
Name: Total_Stops, dtype: int64

In [446]:
date_of_journey = pd.to_datetime(all_data['Date_of_Journey'],format="%d/%m/%Y")
all_data['Month_of_Journey'] = date_of_journey.dt.month
all_data['Day_of_week'] = date_of_journey.dt.dayofweek
all_data['Day_of_month'] = date_of_journey.dt.day

In [447]:
duration = all_data.Duration.str.replace(r'h','')
duration = duration.str.replace(r'm','')
duration = duration.apply(lambda x: str(x).split(" "))

In [448]:
all_data['Duration_in_minutes'] = duration.apply(lambda x : (int(x[0])*60)+int(x[-1]))
#duration.apply(lambda x: x[-1])

In [452]:
all_data['Dep_Hour'] = pd.to_datetime(all_data['Dep_Time']).dt.hour
all_data['Dep_minute'] = pd.to_datetime(all_data['Dep_Time']).dt.minute

In [453]:
all_data['Duration_in_hours'] = duration.apply(lambda x : (int(x[0]))+int(x[-1])/60)

In [454]:
all_data = pd.concat([all_data,pd.get_dummies(all_data['Airline'])], axis=1)
all_data = pd.concat([all_data,pd.get_dummies(all_data['Source'])], axis=1)
all_data = pd.concat([all_data,pd.get_dummies(all_data['Destination'])], axis=1)
#all_data = pd.concat([all_data,pd.get_dummies(all_data['Route'])], axis=1)


In [455]:
all_data = pd.concat([all_data,pd.get_dummies(all_data['Additional_Info'])], axis=1)

In [ ]:
all_data['Route'].str.decode("utf-8", "strict").get_dummies(sep=' \u2192 ')

In [476]:
banglore = all_data['Banglore'].sum(axis=1)
kolkata =  all_data['Kolkata'].sum(axis=1)
delhi =    all_data['Delhi'].sum(axis=1)+all_data['New Delhi']

In [480]:
del all_data['Banglore']
del all_data['Kolkata']
del all_data['Delhi']

In [485]:
del all_data['New Delhi']

In [486]:
all_data.columns

Index([                          u'Airline',
                         u'Date_of_Journey',
                                  u'Source',
                             u'Destination',
                                   u'Route',
                                u'Dep_Time',
                            u'Arrival_Time',
                                u'Duration',
                             u'Total_Stops',
                         u'Additional_Info',
                        u'Month_of_Journey',
                             u'Day_of_week',
                            u'Day_of_month',
                     u'Duration_in_minutes',
                                u'Dep_Hour',
                              u'Dep_minute',
                       u'Duration_in_hours',
                                u'Air Asia',
                               u'Air India',
                                   u'GoAir',
                                  u'IndiGo',
                             u'Jet Airways',
          

In [534]:
pd.concat([all_data.loc[0:10682],target],axis=1).to_csv('train.csv',encoding='utf-8')
all_data.loc[10683:].to_csv('test.csv',encoding='utf-8')

In [487]:
all_data['Banglore'] = banglore
all_data['Kolkata']  = kolkata
all_data['Delhi']    = delhi

In [488]:
all_data['No_info'] = all_data['No Info']+all_data['No info']
del all_data['No Info']
del all_data['No info']

In [489]:
del all_data['Airline'] 
del all_data['Destination']
del all_data['Source']
del all_data['Route']
del all_data['Additional_Info']
del all_data['Date_of_Journey']
del all_data['Dep_Time']
del all_data['Arrival_Time']
del all_data['Duration']

In [490]:
train = all_data.loc[0:10682]

In [491]:
test = all_data.loc[10683:]

In [492]:
test_data.head(10)

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info
0,Jet Airways,6/06/2019,Delhi,Cochin,DEL → BOM → COK,17:30,04:25 07 Jun,10h 55m,1 stop,No info
1,IndiGo,12/05/2019,Kolkata,Banglore,CCU → MAA → BLR,06:20,10:20,4h,1 stop,No info
2,Jet Airways,21/05/2019,Delhi,Cochin,DEL → BOM → COK,19:15,19:00 22 May,23h 45m,1 stop,In-flight meal not included
3,Multiple carriers,21/05/2019,Delhi,Cochin,DEL → BOM → COK,08:00,21:00,13h,1 stop,No info
4,Air Asia,24/06/2019,Banglore,Delhi,BLR → DEL,23:55,02:45 25 Jun,2h 50m,non-stop,No info
5,Jet Airways,12/06/2019,Delhi,Cochin,DEL → BOM → COK,18:15,12:35 13 Jun,18h 20m,1 stop,In-flight meal not included
6,Air India,12/03/2019,Banglore,New Delhi,BLR → TRV → DEL,07:30,22:35,15h 5m,1 stop,No info
7,IndiGo,1/05/2019,Kolkata,Banglore,CCU → HYD → BLR,15:15,20:30,5h 15m,1 stop,No info
8,IndiGo,15/03/2019,Kolkata,Banglore,CCU → BLR,10:10,12:55,2h 45m,non-stop,No info
9,Jet Airways,18/05/2019,Kolkata,Banglore,CCU → BOM → BLR,16:30,22:35,6h 5m,1 stop,No info


In [493]:
test.head(10)

,Total_Stops,Month_of_Journey,Day_of_week,Day_of_month,Duration_in_minutes,Dep_Hour,Dep_minute,Duration_in_hours,Air Asia,Air India,...,2 Long layover,Business class,Change airports,In-flight meal not included,No check-in baggage included,Red-eye flight,Banglore,Kolkata,Delhi,No_info
10683,1,6,3,6,33000,17,30,10,0,0,...,0,0,0,0,0,0,0,0,1,1
10684,1,5,6,12,960,6,20,4,0,0,...,0,0,0,0,0,0,1,1,0,1
10685,1,5,1,21,62100,19,15,23,0,0,...,0,0,0,1,0,0,0,0,1,0
10686,1,5,1,21,10140,8,0,13,0,0,...,0,0,0,0,0,0,0,0,1,1
10687,0,6,0,24,6000,23,55,2,1,0,...,0,0,0,0,0,0,1,0,1,1
10688,1,6,2,12,21600,18,15,18,0,0,...,0,0,0,1,0,0,0,0,1,0
10689,1,3,1,12,4500,7,30,15,0,1,...,0,0,0,0,0,0,1,0,1,1
10690,1,5,2,1,4500,15,15,5,0,0,...,0,0,0,0,0,0,1,1,0,1
10691,0,3,4,15,5400,10,10,2,0,0,...,0,0,0,0,0,0,1,1,0,1
10692,1,5,5,18,1800,16,30,6,0,0,...,0,0,0,0,0,0,1,1,0,1


In [494]:
from sklearn.model_selection import KFold,cross_val_score
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor,BaggingRegressor,AdaBoostRegressor,ExtraTreesRegressor

In [27]:
from modAL.models import ActiveLearner
from modAL.uncertainty import uncertainty_sampling
from sklearn.ensemble import RandomForestClassifier

In [ ]:
learner = ActiveLearner(
    estimator=RandomForestClassifier(),
    query_strategy=uncertainty_sampling,
    X_training=train, y_training=target
)

/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [495]:

def rmsle_cv(model):
    kf = KFold(n_splits=5, shuffle=True, random_state=42).get_n_splits(train.values)
    rmse= np.sqrt(-cross_val_score(model, train.values, target.values, scoring="neg_mean_squared_log_error", cv = kf))
    print(rmse)
    return(rmse)




In [256]:
xgbr =XGBRegressor()
rfr = RandomForestRegressor()

In [257]:
score = rmsle_cv(xgbr)
print(score.mean(), score.std())

[0.19926334 0.19065823 0.19773182 0.19716    0.19993614]
(0.19694990317093888, 0.0033018265230220644)


In [496]:
adbr = AdaBoostRegressor()
rmsle_cv(adbr)

[0.45402933 0.49784847 0.4422841  0.39371736 0.35664685]


array([0.45402933, 0.49784847, 0.4422841 , 0.39371736, 0.35664685])

In [497]:
gbr = GradientBoostingRegressor(learning_rate=0.05)
rmsle_cv(gbr)

[0.21083082 0.20126303 0.20892074 0.21278338 0.21048807]


array([0.21083082, 0.20126303, 0.20892074, 0.21278338, 0.21048807])

In [301]:
from sklearn.tree import DecisionTreeRegressor

In [498]:
dsr = DecisionTreeRegressor()
rmsle_cv(dsr)

[0.17099841 0.15602334 0.16007412 0.16363922 0.16811268]


array([0.17099841, 0.15602334, 0.16007412, 0.16363922, 0.16811268])

### stacking 1

In [185]:
from lightgbm import LGBMRegressor

In [499]:
lgbm = LGBMRegressor(learning_rate=0.4,n_estimators=200,n_jobs=-1)
scorelgbm = rmsle_cv(lgbm)

[0.13685029 0.14055867 0.1463728  0.15575614 0.14437113]


### stacking 2

In [500]:
bgr = BaggingRegressor(n_estimators=350)
rmsle_cv(bgr)

[0.12797694 0.12388892 0.12572603 0.1292796  0.13335455]


array([0.12797694, 0.12388892, 0.12572603, 0.1292796 , 0.13335455])

In [507]:
bgr = BaggingRegressor(n_estimators=500)
rmsle_cv(bgr)

[0.12794063 0.12393327 0.12522483 0.12853562 0.13262805]


array([0.12794063, 0.12393327, 0.12522483, 0.12853562, 0.13262805])

In [511]:
bgr = BaggingRegressor(n_estimators=550)
rmsle_cv(bgr)

[0.12736933 0.12518295 0.12541255 0.12876404 0.13318844]


array([0.12736933, 0.12518295, 0.12541255, 0.12876404, 0.13318844])

In [508]:
bgr = BaggingRegressor(n_estimators=450)
rmsle_cv(bgr)

[0.12741932 0.12398644 0.12520014 0.12899446 0.13318207]


array([0.12741932, 0.12398644, 0.12520014, 0.12899446, 0.13318207])

In [502]:
bgr.fit(train.values,target.values)
bgrpred = bgr.predict(test.values)

In [503]:
pd.DataFrame({'Price':bgrpred}).to_excel('submission4.xlsx',index=False)

### stacking 3

In [504]:
xtr = ExtraTreesRegressor(n_estimators=100)
rmsle_cv(xtr)

[0.12882031 0.12953028 0.12678643 0.13318001 0.13017943]


array([0.12882031, 0.12953028, 0.12678643, 0.13318001, 0.13017943])

In [505]:
xtr = ExtraTreesRegressor(n_estimators=150)
rmsle_cv(xtr)

[0.12771004 0.13092525 0.1257294  0.13183547 0.13036073]


array([0.12771004, 0.13092525, 0.1257294 , 0.13183547, 0.13036073])

In [520]:
estimators = [('bgr1',BaggingRegressor(n_estimators=500)),
              ('bgr2',BaggingRegressor(n_estimators=450))]
              
# Initialize StackingTransformer
stack = StackingTransformer(estimators, regression=True, verbose=2)

# Fit
stack = stack.fit(train.values, target.values)

task:         [regression]
metric:       [mean_absolute_error]
variant:      [A]
n_estimators: [2]

estimator  0: [bgr1: BaggingRegressor]
    fold  0:  [683.15134026]
    fold  1:  [659.24989871]
    fold  2:  [658.96810212]
    fold  3:  [673.80652058]
    ----
    MEAN:     [668.79396542] + [10.23348328]

estimator  1: [bgr2: BaggingRegressor]
    fold  0:  [682.90886072]
    fold  1:  [660.84347646]
    fold  2:  [658.57680884]
    fold  3:  [672.84231021]
    ----
    MEAN:     [668.79286406] + [9.78800188]



In [521]:
S_train = stack.transform(train.values)
S_test = stack.transform(test.values)

lgbms = LGBMRegressor(boosting_type='gbdt')
lgbms.fit(S_train,target.values)
predlgbm = lgbms.predict(S_test)

xgbrs = XGBRegressor(objective ='reg:linear', learning_rate = 0.0151,n_estimators = 130,max_depth=3,
                     min_child_weight=8)
xgbrs.fit(S_train,target.values)
predxgbrs = xgbrs.predict(S_test)


lrs = LinearRegression(normalize=True)
br =BayesianRidge(n_iter=500,normalize=True)

def rmsle_stack_cv(model):
    kf = KFold(n_splits=5, shuffle=True, random_state=42).get_n_splits(S_train)
    rmse= np.sqrt(-cross_val_score(model, S_train, target.values, scoring="neg_mean_squared_log_error", cv = kf))
    print(rmse)
    return(rmse)

rmsle_stack_cv(lgbms)
rmsle_stack_cv(xgbrs)
rmsle_stack_cv(lrs)

rmsle_stack_cv(lrs)

rmsle_stack_cv(br)
rmsle_stack_cv(hr)
rmsle_stack_cv(par)
rmsle_stack_cv(dsr)
rmsle_stack_cv(ran)
rmsle_stack_cv(knnr)
rmsle_stack_cv(svrr)

Train set was detected.
Transforming...

estimator  0: [bgr1: BaggingRegressor]
    model from fold  0: done
    model from fold  1: done
    model from fold  2: done
    model from fold  3: done
    ----
    DONE

estimator  1: [bgr2: BaggingRegressor]
    model from fold  0: done
    model from fold  1: done
    model from fold  2: done
    model from fold  3: done
    ----
    DONE

Transforming...

estimator  0: [bgr1: BaggingRegressor]
    model from fold  0: done
    model from fold  1: done
    model from fold  2: done
    model from fold  3: done
    ----
    DONE

estimator  1: [bgr2: BaggingRegressor]
    model from fold  0: done
    model from fold  1: done
    model from fold  2: done
    model from fold  3: done
    ----
    DONE

[0.12873751 0.13164944 0.13058693 0.13260174 0.13586348]
[0.1909876  0.18927459 0.185571   0.19571671 0.19183559]
[0.12654675 0.12786077 0.12655585 0.1302346  0.13280506]
[0.12654675 0.12786077 0.12655585 0.1302346  0.13280506]
[0.12652297 0.1278

array([0.51970832, 0.51459915, 0.51839235, 0.50785306, 0.51722599])

In [522]:
bgrs = BaggingRegressor()
rmsle_stack_cv(xtr)

[0.14854009 0.14843068 0.14072305 0.14679731 0.14788934]


array([0.14854009, 0.14843068, 0.14072305, 0.14679731, 0.14788934])

In [533]:
rmsle_stack_cv(MLPRegressor(learning_rate_init=0.0001))

[0.12671311 0.12770195 0.12642996 0.13014755 0.13291028]


array([0.12671311, 0.12770195, 0.12642996, 0.13014755, 0.13291028])

### stacking 4

In [506]:
score = rmsle_cv(rfr)
print(score.mean(), score.std())

[0.13363356 0.13444123 0.13281304 0.13633338 0.13554096]
(0.13455243268759895, 0.0012673732045111092)


In [259]:
rfr.fit(train.values, target.values)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [260]:
rfrpred = rfr.predict(test.values)

pd.DataFrame({'Price':rfrpred}).to_excel('submission2.xlsx',index=False)

### Randomized search for best hyper parameters

In [339]:
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
random_grid_bag = {'n_estimators': n_estimators,
               }
print(random_grid)

{'bootstrap': [True, False], 'min_samples_leaf': [1, 2, 4], 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'min_samples_split': [2, 5, 10], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None]}


In [340]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
bgrcv = BaggingRegressor()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
bgr_random = RandomizedSearchCV(estimator = bgrcv, param_distributions = random_grid_bag, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
bgr_random.fit(train.values, target.values)

/anaconda3/lib/python2.7/site-packages/sklearn/model_selection/_search.py:271: UserWarning: The total space of parameters 10 is smaller than n_iter=100. Running 10 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Fitting 3 folds for each of 10 candidates, totalling 30 fits


/anaconda3/lib/python2.7/site-packages/sklearn/externals/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:  3.8min finished


RandomizedSearchCV(cv=3, error_score='raise-deprecating',
          estimator=BaggingRegressor(base_estimator=None, bootstrap=True,
         bootstrap_features=False, max_features=1.0, max_samples=1.0,
         n_estimators=10, n_jobs=None, oob_score=False, random_state=None,
         verbose=0, warm_start=False),
          fit_params=None, iid='warn', n_iter=100, n_jobs=-1,
          param_distributions={'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]},
          pre_dispatch='2*n_jobs', random_state=42, refit=True,
          return_train_score='warn', scoring=None, verbose=2)

In [315]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestRegressor()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(train.values, target.values)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
/anaconda3/lib/python2.7/site-packages/sklearn/externals/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
/anaconda3/lib/python2.7/site-packages/sklearn/externals/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
/anaconda3/lib/python2.7/site-packages/sklearn/externals/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
/anaconda3/lib/python2.7/site-packages/skl

/anaconda3/lib/python2.7/site-packages/sklearn/externals/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
/anaconda3/lib/python2.7/site-packages/sklearn/externals/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
/anaconda3/lib/python2.7/site-packages/sklearn/externals/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
/anaconda3/lib/python2.7/site-packages/sklearn/externals/joblib/externals/loky/process_executor.py:700: UserWarning: A

RandomizedSearchCV(cv=3, error_score='raise-deprecating',
          estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False),
          fit_params=None, iid='warn', n_iter=100, n_jobs=-1,
          param_distributions={'bootstrap': [True, False], 'min_samples_leaf': [1, 2, 4], 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'min_samples_split': [2, 5, 10], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None]},
          pre_dispatch='2*n_jobs', random_state=42, refit=True,
          return_train_score='warn', scoring=None, verbose=2)

In [326]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
lgbmcv = LGBMRegressor(learning_rate=0.4,n_estimators=200,n_jobs=-1)
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
lgbm_random = RandomizedSearchCV(estimator = lgbmcv, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
lgbm_random.fit(train.values, target.values)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   26.1s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  3.2min finished


RandomizedSearchCV(cv=3, error_score='raise-deprecating',
          estimator=LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
       importance_type='split', learning_rate=0.4, max_depth=-1,
       min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
       n_estimators=200, n_jobs=-1, num_leaves=31, objective=None,
       random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
       subsample=1.0, subsample_for_bin=200000, subsample_freq=0),
          fit_params=None, iid='warn', n_iter=100, n_jobs=-1,
          param_distributions={'bootstrap': [True, False], 'min_samples_leaf': [1, 2, 4], 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'min_samples_split': [2, 5, 10], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None]},
          pre_dispatch='2*n_jobs', random_state=42, refit=True,
          return_train_score='warn', scoring=None, verbose=2)

In [327]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
xtrcv = ExtraTreesRegressor(n_estimators=100)
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
xtr_random = RandomizedSearchCV(estimator = xtrcv, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
xtr_random.fit(train.values, target.values)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
/anaconda3/lib/python2.7/site-packages/sklearn/externals/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
/anaconda3/lib/python2.7/site-packages/sklearn/externals/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
/anaconda3/lib/python2.7/site-packages/sklearn/externals/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
/anaconda3/lib/python2.7/site-packages/skl

/anaconda3/lib/python2.7/site-packages/sklearn/externals/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
/anaconda3/lib/python2.7/site-packages/sklearn/externals/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
/anaconda3/lib/python2.7/site-packages/sklearn/externals/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
/anaconda3/lib/python2.7/site-packages/sklearn/externals/joblib/externals/loky/process_executor.py:700: UserWarning: A

RandomizedSearchCV(cv=3, error_score='raise-deprecating',
          estimator=ExtraTreesRegressor(bootstrap=False, criterion='mse', max_depth=None,
          max_features='auto', max_leaf_nodes=None,
          min_impurity_decrease=0.0, min_impurity_split=None,
          min_samples_leaf=1, min_samples_split=2,
          min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
          oob_score=False, random_state=None, verbose=0, warm_start=False),
          fit_params=None, iid='warn', n_iter=100, n_jobs=-1,
          param_distributions={'bootstrap': [True, False], 'min_samples_leaf': [1, 2, 4], 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'min_samples_split': [2, 5, 10], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None]},
          pre_dispatch='2*n_jobs', random_state=42, refit=True,
          return_train_score='warn', scoring=None, verbose=2)

In [328]:
xtr_random.best_params_

{'bootstrap': False,
 'max_depth': 90,
 'max_features': 'auto',
 'min_samples_leaf': 2,
 'min_samples_split': 5,
 'n_estimators': 200}

In [329]:
lgbm_random.best_params_

{'bootstrap': True,
 'max_depth': 10,
 'max_features': 'auto',
 'min_samples_leaf': 4,
 'min_samples_split': 5,
 'n_estimators': 200}

In [330]:
rf_random.best_params_

{'bootstrap': True,
 'max_depth': 100,
 'max_features': 'auto',
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'n_estimators': 1400}

In [341]:
bgr_random.best_params_

{'n_estimators': 600}

In [331]:
best_random = rf_random.best_estimator_
rmsle_cv(best_random)
#random_accuracy = evaluate(best_random, df_train, df_target)

[0.13119604 0.12839829 0.12771084 0.13388431 0.13366113]


array([0.13119604, 0.12839829, 0.12771084, 0.13388431, 0.13366113])

In [332]:
best_lgbmr = lgbm_random.best_estimator_
rmsle_cv(best_lgbmr)

[0.14047252 0.14651586 0.14512022 0.14847994 0.14443236]


array([0.14047252, 0.14651586, 0.14512022, 0.14847994, 0.14443236])

In [333]:
best_xtr = xtr_random.best_estimator_
rmsle_cv(best_xtr)

[0.13155389 0.12924224 0.12893024 0.13220022 0.13222393]


array([0.13155389, 0.12924224, 0.12893024, 0.13220022, 0.13222393])

In [342]:
best_bgr = bgr_random.best_estimator_
rmsle_cv(best_bgr)

[0.13139281 0.12858758 0.12742066 0.13447115 0.13394666]


array([0.13139281, 0.12858758, 0.12742066, 0.13447115, 0.13394666])

In [343]:
from sklearn.neural_network import MLPRegressor
mlpr = MLPRegressor()
rmsle_cv(mlpr)

/anaconda3/lib/python2.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/anaconda3/lib/python2.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/anaconda3/lib/python2.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/anaconda3/lib/python2.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


[0.37800354 0.39177923 0.42690135 0.38834182 0.4056734 ]


/anaconda3/lib/python2.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


array([0.37800354, 0.39177923, 0.42690135, 0.38834182, 0.4056734 ])

### Stacking

In [348]:
from vecstack import StackingTransformer

# Get your data

# Initialize 1st level estimators
estimators = [('rf', best_random),
              ('br', best_bgr),
              ('xtr',best_xtr)]
              
# Initialize StackingTransformer
stack = StackingTransformer(estimators, regression=True, verbose=2)

# Fit
stack = stack.fit(train.values, target.values)


task:         [regression]
metric:       [mean_absolute_error]
variant:      [A]
n_estimators: [3]

estimator  0: [rf: RandomForestRegressor]
    fold  0:  [715.54363362]
    fold  1:  [673.15793723]
    fold  2:  [701.45833548]
    fold  3:  [681.00207734]
    ----
    MEAN:     [692.79049592] + [16.71257679]

estimator  1: [br: BaggingRegressor]
    fold  0:  [717.23808949]
    fold  1:  [672.90045445]
    fold  2:  [703.04626260]
    fold  3:  [676.84857531]
    ----
    MEAN:     [692.50834546] + [18.38685488]

estimator  2: [xtr: ExtraTreesRegressor]
    fold  0:  [732.07954378]
    fold  1:  [685.85201033]
    fold  2:  [717.48937665]
    fold  3:  [671.04207052]
    ----
    MEAN:     [701.61575032] + [24.30668694]



In [349]:
S_train = stack.transform(train.values)
S_test = stack.transform(test.values)

Train set was detected.
Transforming...

estimator  0: [rf: RandomForestRegressor]
    model from fold  0: done
    model from fold  1: done
    model from fold  2: done
    model from fold  3: done
    ----
    DONE

estimator  1: [br: BaggingRegressor]
    model from fold  0: done
    model from fold  1: done
    model from fold  2: done
    model from fold  3: done
    ----
    DONE

estimator  2: [xtr: ExtraTreesRegressor]
    model from fold  0: done
    model from fold  1: done
    model from fold  2: done
    model from fold  3: done
    ----
    DONE

Transforming...

estimator  0: [rf: RandomForestRegressor]
    model from fold  0: done
    model from fold  1: done
    model from fold  2: done
    model from fold  3: done
    ----
    DONE

estimator  1: [br: BaggingRegressor]
    model from fold  0: done
    model from fold  1: done
    model from fold  2: done
    model from fold  3: done
    ----
    DONE

estimator  2: [xtr: ExtraTreesRegressor]
    model from fold  0: don

In [352]:
lgbms = LGBMRegressor(boosting_type='gbdt')
lgbms.fit(S_train,target.values)
predlgbm = lgbms.predict(S_test)

xgbrs = XGBRegressor(objective ='reg:linear', learning_rate = 0.0151,n_estimators = 130,max_depth=3,
                     min_child_weight=8)
xgbrs.fit(S_train,target.values)
predxgbrs = xgbrs.predict(S_test)


In [359]:
lrs = LinearRegression(normalize=True)

In [364]:
from sklearn.linear_model import BayesianRidge, 

In [369]:
br =BayesianRidge(n_iter=500,normalize=True)

In [355]:
def rmsle_stack_cv(model):
    kf = KFold(n_splits=5, shuffle=True, random_state=42).get_n_splits(S_train)
    rmse= np.sqrt(-cross_val_score(model, S_train, target.values, scoring="neg_mean_squared_log_error", cv = kf))
    print(rmse)
    return(rmse)

In [358]:
rmsle_stack_cv(lgbms)
rmsle_stack_cv(xgbrs)
rmsle_stack_cv(lrs)

[0.13190024 0.13475336 0.13361986 0.1398382  0.13594309]
[0.19275111 0.19184912 0.18710733 0.19820683 0.19151963]
[0.12974485 0.13117148 0.13036315 0.13340794 0.13144961]


array([0.12974485, 0.13117148, 0.13036315, 0.13340794, 0.13144961])

In [360]:
rmsle_stack_cv(lrs)

[0.12974485 0.13117148 0.13036315 0.13340794 0.13144961]


array([0.12974485, 0.13117148, 0.13036315, 0.13340794, 0.13144961])

In [373]:
rmsle_stack_cv(br)
rmsle_stack_cv(hr)
rmsle_stack_cv(par)
rmsle_stack_cv(dsr)
rmsle_stack_cv(ran)
rmsle_stack_cv(knnr)
rmsle_stack_cv(svrr)

[0.12973107 0.13112358 0.13027507 0.13336476 0.13147703]
[0.1300138  0.13091019 0.12947867 0.13317887 0.1320302 ]
[0.13340956 0.13047208 0.12937635 0.13415421 0.1369087 ]
[0.17547135 0.1870353  0.19042605 0.19313901 0.18618287]
[0.13141864 0.13223009 0.13146493 0.13766535 0.13888553]
[0.13771512 0.14160354 0.14510284 0.14414455 0.14272767]
[0.51973597 0.51462416 0.51842381 0.50788696 0.51725097]


array([0.51973597, 0.51462416, 0.51842381, 0.50788696, 0.51725097])

In [377]:
hr.fit(S_train,target.values)
pd.DataFrame({'Price':hr.predict(S_test)}).to_excel('stacking_sub1.xlsx', index=False)

In [312]:
grid_search.best_estimator_

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
       importance_type='split', learning_rate=0.4, max_depth=-1,
       min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
       n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
       random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
       subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [206]:
data.Source.unique()

array([u'Banglore', u'Kolkata', u'Delhi', u'Chennai', u'Mumbai'],
      dtype=object)

In [ ]:
data.data.Sourceourceourceourceourceource

In [207]:
test_data.Source.unique()

array([u'Delhi', u'Kolkata', u'Banglore', u'Mumbai', u'Chennai'],
      dtype=object)

In [279]:
from sklearn.svm import SVR
svrr = SVR()
rmsle_cv(svrr)


/anaconda3/lib/python2.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


[0.51925767 0.51415282 0.51794674 0.50736465 0.51676091]


array([0.51925767, 0.51415282, 0.51794674, 0.50736465, 0.51676091])

In [286]:
from sklearn.linear_model import ARDRegression,HuberRegressor,PassiveAggressiveRegressor,RANSACRegressor,SGDRegressor,LinearRegression

In [283]:
hr = HuberRegressor()
rmsle_cv(hr)

[0.45998928 0.38936658 0.45201298 0.446718   0.3959029 ]


array([0.45998928, 0.38936658, 0.45201298, 0.446718  , 0.3959029 ])

In [284]:
ran = RANSACRegressor()
rmsle_cv(ran)

/anaconda3/lib/python2.7/site-packages/scipy/linalg/basic.py:1018: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  warnings.warn(mesg, RuntimeWarning)


[0.28912191 0.2716972  0.27648916 0.27626035 0.26609591]


array([0.28912191, 0.2716972 , 0.27648916, 0.27626035, 0.26609591])

In [285]:
par = PassiveAggressiveRegressor()
rmsle_cv(par)

[0.58612067 0.57678665 0.82213766 1.48608108 0.57820205]


/anaconda3/lib/python2.7/site-packages/sklearn/linear_model/stochastic_gradient.py:144: FutureWarning: max_iter and tol parameters have been added in PassiveAggressiveRegressor in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


array([0.58612067, 0.57678665, 0.82213766, 1.48608108, 0.57820205])

In [287]:
lr = LinearRegression()
rmsle_cv(lr)

[0.26724582 0.25815018 0.2722209  0.2716096  0.26457948]


array([0.26724582, 0.25815018, 0.2722209 , 0.2716096 , 0.26457948])

In [288]:
from sklearn.neighbors import KNeighborsRegressor

In [289]:
knnr = KNeighborsRegressor()
rmsle_cv(knnr)

[0.30319623 0.29933687 0.298964   0.30472291 0.29222781]


array([0.30319623, 0.29933687, 0.298964  , 0.30472291, 0.29222781])

In [413]:
hack = pd.read_excel('stacking_sub1.xlsx')

In [414]:
hack.loc[(test_data['Source']=='Mumbai') & (test_data['Destination']=='Hyderabad') &
     (test_data['Airline']=='SpiceJet') & (test_data['Total_Stops'] == 'non-stop'),'Price'] = 1970

In [415]:
test_data[(test_data['Source']=='Mumbai') & (test_data['Destination']=='Hyderabad') &
     (test_data['Airline']=='Air India') & (test_data['Total_Stops'] == 'non-stop') & 
          (test_data['Additional_Info']=='No info')]

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info
47,Air India,3/05/2019,Mumbai,Hyderabad,BOM → HYD,13:55,15:25,1h 30m,non-stop,No info
94,Air India,27/06/2019,Mumbai,Hyderabad,BOM → HYD,19:30,20:45,1h 15m,non-stop,No info
95,Air India,15/05/2019,Mumbai,Hyderabad,BOM → HYD,06:20,07:40,1h 20m,non-stop,No info
314,Air India,3/03/2019,Mumbai,Hyderabad,BOM → HYD,15:00,16:25,1h 25m,non-stop,No info
343,Air India,18/06/2019,Mumbai,Hyderabad,BOM → HYD,21:05,22:25,1h 20m,non-stop,No info
769,Air India,21/05/2019,Mumbai,Hyderabad,BOM → HYD,06:20,07:40,1h 20m,non-stop,No info
797,Air India,15/03/2019,Mumbai,Hyderabad,BOM → HYD,13:55,15:25,1h 30m,non-stop,No info
836,Air India,15/05/2019,Mumbai,Hyderabad,BOM → HYD,13:55,15:25,1h 30m,non-stop,No info
908,Air India,24/06/2019,Mumbai,Hyderabad,BOM → HYD,19:30,20:45,1h 15m,non-stop,No info
973,Air India,9/05/2019,Mumbai,Hyderabad,BOM → HYD,06:20,07:40,1h 20m,non-stop,No info


In [416]:
test_data[(test_data['Source']=='Mumbai') & (test_data['Destination']=='Hyderabad') &
     (test_data['Airline']=='Jet Airways') & (test_data['Total_Stops'] == 'non-stop') & 
          (test_data['Additional_Info']=='In-flight meal not included')]

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info
34,Jet Airways,15/03/2019,Mumbai,Hyderabad,BOM → HYD,07:10,08:35,1h 25m,non-stop,In-flight meal not included
159,Jet Airways,12/05/2019,Mumbai,Hyderabad,BOM → HYD,19:35,21:05,1h 30m,non-stop,In-flight meal not included
307,Jet Airways,21/03/2019,Mumbai,Hyderabad,BOM → HYD,15:50,17:20,1h 30m,non-stop,In-flight meal not included
362,Jet Airways,3/05/2019,Mumbai,Hyderabad,BOM → HYD,19:35,21:05,1h 30m,non-stop,In-flight meal not included
425,Jet Airways,27/06/2019,Mumbai,Hyderabad,BOM → HYD,19:35,21:05,1h 30m,non-stop,In-flight meal not included
996,Jet Airways,21/03/2019,Mumbai,Hyderabad,BOM → HYD,10:20,11:50,1h 30m,non-stop,In-flight meal not included
1078,Jet Airways,1/04/2019,Mumbai,Hyderabad,BOM → HYD,07:05,08:30,1h 25m,non-stop,In-flight meal not included
1157,Jet Airways,1/04/2019,Mumbai,Hyderabad,BOM → HYD,19:35,21:05,1h 30m,non-stop,In-flight meal not included
1372,Jet Airways,1/05/2019,Mumbai,Hyderabad,BOM → HYD,10:20,11:50,1h 30m,non-stop,In-flight meal not included
1580,Jet Airways,15/06/2019,Mumbai,Hyderabad,BOM → HYD,02:55,04:20,1h 25m,non-stop,In-flight meal not included


In [417]:
hack.loc[(test_data['Source']=='Mumbai') & (test_data['Destination']=='Hyderabad') &
     (test_data['Airline']=='Air India') & (test_data['Total_Stops'] == 'non-stop') &
       (test_data['Additional_Info']=='No info'),'Price'] = 3100

In [418]:
test_data[(test_data['Source']=='Mumbai') & (test_data['Destination']=='Hyderabad') &
     (test_data['Airline']=='SpiceJet') & (test_data['Total_Stops'] == 'non-stop') &
         (test_data['Additional_Info']== 'No check-in baggage included')]

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info
1049,SpiceJet,15/05/2019,Mumbai,Hyderabad,BOM → HYD,05:45,07:15,1h 30m,non-stop,No check-in baggage included
1051,SpiceJet,6/05/2019,Mumbai,Hyderabad,BOM → HYD,13:15,14:45,1h 30m,non-stop,No check-in baggage included
1147,SpiceJet,1/06/2019,Mumbai,Hyderabad,BOM → HYD,05:45,07:15,1h 30m,non-stop,No check-in baggage included
1291,SpiceJet,27/05/2019,Mumbai,Hyderabad,BOM → HYD,05:45,07:15,1h 30m,non-stop,No check-in baggage included
1401,SpiceJet,3/05/2019,Mumbai,Hyderabad,BOM → HYD,22:45,00:15 04 May,1h 30m,non-stop,No check-in baggage included
1572,SpiceJet,27/03/2019,Mumbai,Hyderabad,BOM → HYD,13:10,14:30,1h 20m,non-stop,No check-in baggage included
1588,SpiceJet,24/06/2019,Mumbai,Hyderabad,BOM → HYD,05:45,07:15,1h 30m,non-stop,No check-in baggage included
1624,SpiceJet,3/06/2019,Mumbai,Hyderabad,BOM → HYD,05:45,07:15,1h 30m,non-stop,No check-in baggage included
1800,SpiceJet,9/06/2019,Mumbai,Hyderabad,BOM → HYD,13:15,14:45,1h 30m,non-stop,No check-in baggage included
2058,SpiceJet,1/05/2019,Mumbai,Hyderabad,BOM → HYD,05:45,07:15,1h 30m,non-stop,No check-in baggage included


In [419]:
hack.loc[(test_data['Source']=='Mumbai') & (test_data['Destination']=='Hyderabad') &
     (test_data['Airline']=='SpiceJet') & (test_data['Total_Stops'] == 'non-stop') &
         (test_data['Additional_Info']== 'No check-in baggage included'),'Price'] = 2175

In [420]:
test_data[(test_data['Source']=='Mumbai') & (test_data['Destination']=='Hyderabad') &
     (test_data['Airline']=='IndiGo') & (test_data['Total_Stops'] == 'non-stop') & 
          (test_data['Additional_Info']=='No info')]

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info
70,IndiGo,18/06/2019,Mumbai,Hyderabad,BOM → HYD,12:00,13:30,1h 30m,non-stop,No info
74,IndiGo,18/05/2019,Mumbai,Hyderabad,BOM → HYD,09:10,10:40,1h 30m,non-stop,No info
106,IndiGo,9/05/2019,Mumbai,Hyderabad,BOM → HYD,21:20,22:50,1h 30m,non-stop,No info
181,IndiGo,3/05/2019,Mumbai,Hyderabad,BOM → HYD,19:05,20:35,1h 30m,non-stop,No info
227,IndiGo,1/04/2019,Mumbai,Hyderabad,BOM → HYD,12:00,13:30,1h 30m,non-stop,No info
261,IndiGo,27/03/2019,Mumbai,Hyderabad,BOM → HYD,02:30,04:00,1h 30m,non-stop,No info
513,IndiGo,3/05/2019,Mumbai,Hyderabad,BOM → HYD,16:55,18:25,1h 30m,non-stop,No info
526,IndiGo,3/04/2019,Mumbai,Hyderabad,BOM → HYD,20:05,21:40,1h 35m,non-stop,No info
543,IndiGo,21/06/2019,Mumbai,Hyderabad,BOM → HYD,20:05,21:40,1h 35m,non-stop,No info
556,IndiGo,6/06/2019,Mumbai,Hyderabad,BOM → HYD,20:05,21:40,1h 35m,non-stop,No info


In [421]:
hack.loc[(test_data['Source']=='Mumbai') & (test_data['Destination']=='Hyderabad') &
     (test_data['Airline']=='IndiGo') & (test_data['Total_Stops'] == 'non-stop') & 
          (test_data['Additional_Info']=='No info'),'Price'] =2754

In [422]:
hack.to_excel('hack_sub1.xlsx',index=False)